In [1]:
PYTHON_PATHS = [".."]
import sys
for path in PYTHON_PATHS:
    if path not in sys.path:
        sys.path.append(path)

In [2]:
import numpy as np
import os
from tqdm.notebook import tqdm as tqdm
import matplotlib.pyplot as plt

In [3]:
from unsupkeypoints.data import DescriptorDataModule
from unsupkeypoints.models import PointNetwork
from unsupkeypoints.criterions import RegressionLoss
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.utilities.parsing import AttributeDict
import torch
import pytorch_lightning as pl

# Data module

In [4]:
ROOT_PATH = os.path.dirname(os.getcwd())
TRAIN_PATH = os.path.join(ROOT_PATH, "data/output_kapture/7scenes/fire/mapping")
TEST_PATH = os.path.join(ROOT_PATH, "data/output_kapture/7scenes/fire/query")

In [5]:
data_module = DescriptorDataModule(TRAIN_PATH, TEST_PATH)

[DescriptorDataModule] - train dataset size 568196
[DescriptorDataModule] - test dataset size 544913


# Model

In [6]:
params = AttributeDict(
    name="point_net",
    optimizer=AttributeDict(),
    hidden_dimensions=(100,),
    input_dimension=512,
)

In [15]:
criterion = RegressionLoss()
model = PointNetwork(params, criterion)
data_module._batch_size = 512

In [16]:
trainer = pl.Trainer(logger=TensorBoardLogger("lightning_logs"), max_epochs=10, gpus=1)

GPU available: True, used: True
INFO    ::lightning: GPU available: True, used: True
TPU available: False, using: 0 TPU cores
INFO    ::lightning: TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO    ::lightning: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [17]:
trainer.fit(model, data_module)


  | Name       | Type           | Params
----------------------------------------------
0 | _criterion | RegressionLoss | 0     
1 | backend    | Sequential     | 51 K  
INFO    ::lightning: 
  | Name       | Type           | Params
----------------------------------------------
0 | _criterion | RegressionLoss | 0     
1 | backend    | Sequential     | 51 K  


1

In [18]:
trainer.test(model, data_module.test_dataloader())

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': tensor(0.1293, device='cuda:0'),
 'train_loss': tensor(0.0638, device='cuda:0'),
 'val_loss': tensor(0.1293, device='cuda:0')}
--------------------------------------------------------------------------------



[{'train_loss': 0.06375681608915329,
  'val_loss': 0.12932470440864563,
  'test_loss': 0.12932470440864563}]

In [20]:
0.06 ** 0.5

0.2449489742783178